In [ ]:
!pip install pprint

In [ ]:
import requests, re, pprint, os

# 自訂標頭
my_headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
}

# 將自訂標頭加入 GET 請求中
response = requests.get('https://store.line.me/stickershop/product/17555/zh-Hant', headers = my_headers)

# 輸出網頁 HTML 原始碼
# print(response.text)


'''
以下會透過兩種方法取得「貼圖 id」與「貼圖連結」
'''
# 建立兩種方法進行儲存圖片的資料夾
folderPath01 = 'line_stickers_finditer'
folderPath02 = 'line_stickers_findall'
if not os.path.exists(folderPath01):
    os.makedirs(folderPath01)
if not os.path.exists(folderPath02):
    os.makedirs(folderPath02)

    

'''
方法一:
透過 re.finditer 變成可迭代物件 (iterable)，再用 for in 取得相關資料
'''
#放貼圖資訊用
listLineStickers = [] 

# 正規表達式
pattern01 = r'https?:\/\/stickershop\.line-scdn\.net\/stickershop\/v1\/sticker\/([0-9]+)\/android\/sticker\.png'

# 透過迭代來整理資料
matchIterable = re.finditer(pattern01, response.text)
for match in matchIterable:
#     print(match.group())
#     print(match.group(1))
    listLineStickers.append({
        "link": match.group(),
        "id": match.group(1)
    })

# 去除重複
listFirstResult = [dict(t) for t in set(tuple(myDict.items()) for myDict in listLineStickers)]

# 依 id 排序
listFirstResult = sorted(listFirstResult, key=lambda myDict: myDict['id'])

# 輸出 listFirstResult 結果
pprint.pprint(listFirstResult)

# 下載圖片
for obj in listFirstResult: 
    os.system(f"curl {obj['link']} -o {folderPath01}/{obj['id']}.png")

    
    
'''
方法二:
將取得 id 與 link 的方法分開變成兩個 list
'''
listLineStickers = [] #放貼圖資訊用

pattern02_1 = r'https?:\/\/stickershop\.line-scdn\.net\/stickershop\/v1\/sticker\/[0-9]+\/android\/sticker\.png'
pattern02_2 = r'https?:\/\/stickershop\.line-scdn\.net\/stickershop\/v1\/sticker\/([0-9]+)\/android\/sticker\.png'

# 取得 link，以 list 格式儲存
listLinks = re.findall(pattern02_1, response.text)

# 取得所有 id，以 list 格式儲存
listIds = re.findall(pattern02_2, response.text)

# 將 listIds 進行排序(由小至大)
listIds = sorted(set(listIds), key=listIds.index)
pprint.pprint(listIds)

# 將 listLinks 進行排序(由小至大)
listLinks = sorted(set(listLinks), key=listLinks.index)
pprint.pprint(listLinks)

# 用 listLinks 的 index，來作為 listIds 的 index 代表的值，作為檔名
for index, link in enumerate(listLinks):
    os.system(f"curl {link} -o {folderPath02}/{listIds[index]}.png")
